In [1]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime
import re
import json
from pandas.api.types import is_numeric_dtype
from simple_salesforce import Salesforce, format_soql, format_external_id
from simple_salesforce.exceptions import SalesforceMalformedRequest
import simpler_sf

simpler_sf.simple_salesforce()

In [2]:
with open("../login.json", "r") as j:
    misDatos = json.load(j)

In [3]:
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)

In [ ]:

    "SELECT Id FROM HealthCloudGA__Program__c WHERE Nombre_Personalizado__c  '3° Básico Matemáticas' AND Paciente__c = '0018c00002anTmeAAE' LIMIT 1"


In [12]:
nombrePersonalizado = "3° Básico Matemáticas"
pacienteId = "0018c00002anTmeAAE"
queryPrograma = (
    "SELECT Id FROM HealthCloudGA__Program__c WHERE Nombre_Personalizado__c  = '"
    + nombrePersonalizado
    + "' AND Paciente__c = '"
    + pacienteId
    + "' LIMIT 1"
)
vllamadaEmpresa = sf.smart_query(queryPrograma)
# vllamadaEmpresa.head()

100%|██████████| 1/1 [00:02<00:00,  2.42s/it]


In [13]:
vllamadaEmpresa.head()

,Id


In [5]:
queryPrograma = (
    "SELECT Id FROM case WHERE Program__c = '" + vllamadaEmpresa["Id"][0] + "'"
)
vllamadaEmpresa = sf.smart_query(queryPrograma)
vllamadaEmpresa.head()
cntdCase = len(vllamadaEmpresa.index)
# print (len(vllamadaEmpresa.index))

IndexError: index 0 is out of bounds for axis 0 with size 0

In [7]:
print(len(vllamadaEmpresa.index))

2


In [8]:
vllamadaEmpresa

,Id
0,500RN000005VKNlYAO
1,500RN000005VKNmYAO


In [9]:
# idGoal2 = pd.DataFrame()
def traerGoal(iD):
    idGoalDF = pd.DataFrame()
    # idGoalTotal = pd.DataFrame()
    idCase = iD
    queryGoal = (
        "SELECT Id FROM HealthCloudGA__CarePlanGoal__c WHERE Etapa__c = '"
        + idCase
        + "'"
    )
    idGoalDF = sf.smart_query(queryGoal)
    # print (len(idGoalDF.index))
    # print (idGoalDF)
    return idGoalDF

In [11]:
idGoal2 = pd.DataFrame()  # Declare idGoal2 as a global variable
derty = pd.DataFrame()

# idGoal2 = vllamadaEmpresa["Id"].apply(lambda y: traerGoal(y))
for registros in range(cntdCase):
    idGoal2 = traerGoal(vllamadaEmpresa["Id"][registros])
    derty = pd.concat([derty, idGoal2], axis=0)
cntdGoal = len(derty.index)
# derty.head(10)

100%|██████████| 1/1 [00:02<00:00,  2.58s/it]


In [12]:
def traerActividad(iD):
    idGoalDF = pd.DataFrame()
    # idGoalTotal = pd.DataFrame()
    idCase = iD
    queryGoal = "SELECT Id FROM Actividad_Programa__c WHERE Goal__c = '" + idCase + "'"
    idGoalDF = sf.smart_query(queryGoal)
    # print (len(idGoalDF.index))
    # print (idGoalDF)
    return idGoalDF

In [13]:
derty.reset_index(drop=True, inplace=True)
derty["Id"][0]

'a0HRN0000013AVJ2A2'

In [14]:
dfActividad = pd.DataFrame()  # Declare idGoal2 as a global variable
dertyActividad = pd.DataFrame()

# idGoal2 = vllamadaEmpresa["Id"].apply(lambda y: traerGoal(y))
for registros in range(cntdGoal):
    dfActividad = traerActividad(derty["Id"][registros])
    dertyActividad = pd.concat([dertyActividad, dfActividad], axis=0)
# cntdGoal = len(derty.index)
dertyActividad.head(10)

100%|██████████| 1/1 [00:03<00:00,  3.22s/it]


,Id
0,a43RN0000001EnJYAU
1,a43RN0000001EnKYAU
0,a43RN0000001EnLYAU
1,a43RN0000001EnMYAU
2,a43RN0000001EnNYAU
3,a43RN0000001EnOYAU
4,a43RN0000001EnPYAU
5,a43RN0000001EnQYAU
6,a43RN0000001EnRYAU
7,a43RN0000001EnSYAU


In [15]:
dertyActividad.reset_index(drop=True, inplace=True)
dertyActividad["Id"][0]
dertyActividad.head()

,Id
0,a43RN0000001EnJYAU
1,a43RN0000001EnKYAU
2,a43RN0000001EnLYAU
3,a43RN0000001EnMYAU
4,a43RN0000001EnNYAU


In [16]:
def GoalCompleto(iD):
    idGoalDF = pd.DataFrame()
    # idGoalTotal = pd.DataFrame()
    idCase = iD
    queryGoal = "SELECT FIELDS(ALL) FROM case WHERE Id = '" + idCase + "'"
    idGoalDF = sf.smart_query(queryGoal)
    # print (len(idGoalDF.index))
    # print (idGoalDF)
    return idGoalDF

In [17]:
goalDF = pd.DataFrame()
goalTotalDF = pd.DataFrame()
for registros in range(len(vllamadaEmpresa.index)):
    goalDF = GoalCompleto(vllamadaEmpresa["Id"][registros])
    goalTotalDF = pd.concat([goalTotalDF, goalDF], axis=0)
# cntdGoal = len(derty.index)

100%|██████████| 1/1 [00:02<00:00,  2.66s/it]


In [18]:
contact = sf.Actividad_Programa__c.get("a43RN0000001EnOYAU")
contact

OrderedDict([('attributes',
              OrderedDict([('type', 'Actividad_Programa__c'),
                           ('url',
                            '/services/data/v42.0/sobjects/Actividad_Programa__c/a43RN0000001EnOYAU')])),
             ('Id', 'a43RN0000001EnOYAU'),
             ('IsDeleted', False),
             ('Name', 'A-0000000132'),
             ('CreatedDate', '2024-01-23T19:33:45.000+0000'),
             ('CreatedById', '0058c00000DSxZmAAL'),
             ('LastModifiedDate', '2024-01-23T19:33:46.000+0000'),
             ('LastModifiedById', '0058c00000DSxZmAAL'),
             ('SystemModstamp', '2024-01-23T19:33:46.000+0000'),
             ('LastActivityDate', None),
             ('LastViewedDate', '2024-01-24T20:38:31.000+0000'),
             ('LastReferencedDate', '2024-01-24T20:38:31.000+0000'),
             ('Goal__c', 'a0HRN0000013AVK2A2'),
             ('Archivo_o_imagen__c', None),
             ('Calcular_puntos_flow__c', False),
             ('Descripcion_Corta_

In [19]:
contact["LastReferencedDate"]

'2024-01-24T20:38:31.000+0000'